<a href="https://colab.research.google.com/github/IrynaBarnych/AI8/blob/main/module3/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%B0_%D1%80%D1%96%D1%88%D0%B5%D0%BD%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [249]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [250]:
df.shape

(1553, 10)

In [251]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [252]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Очистіть дані від викидів(лише `price`), розділіть на тренувальну та тестову чатини

In [253]:
def remove_outliers(df, column_name):
  column = df[column_name]

  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)

  iqr = q3 - q1

  lower = q1 - 1.5*iqr
  upper = q3 + 1.5*iqr

  mask = (column >= lower) & (column <= upper)

  return df[mask]

for column_name in df.columns:
  if df.dtypes[column_name] in ("object", "datetime64[ns]"):
    continue

  df = remove_outliers(df, column_name)

In [254]:
df.shape

(1223, 4)

In [255]:
df['price(in lakhs)'].value_counts()

6.25     17
5.75     14
18.20    14
6.50     14
5.45     12
         ..
5.60      1
1.00      1
5.05      1
3.40      1
11.05     1
Name: price(in lakhs), Length: 507, dtype: int64

In [256]:
y = df['price(in lakhs)']
X = df.drop(columns='price(in lakhs)')

In [257]:
X.head()

,fuel_type,kms_driven,engine(cc)
1,Petrol,30615,999.0
2,Diesel,24000,1995.0
3,Petrol,18378,1353.0
4,Petrol,44900,1798.0
5,Petrol,42000,1199.0


In [258]:
df.dropna(subset=['price(in lakhs)'], inplace=True)

In [259]:
y = df['price(in lakhs)']
X = df.drop(columns='price(in lakhs)')

In [260]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

# Завдання 2
Створіть Pipeline для обробки даних

In [261]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


In [262]:
num_columns = X.select_dtypes(include="number").columns

In [263]:
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )

In [264]:
cat_columns = X.select_dtypes(include="object").columns

In [265]:
cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder())])

In [266]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['fuel_type'], dtype='object'))])

# Завдання 3
Створіть остаточну модель та натренуйте її

In [267]:
preprocessor.fit(X_train)
preprocessor.transform(X_train)

array([[6.1383e+04, 9.9800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [4.5000e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.2000e+05, 1.9950e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       ...,
       [4.0909e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [6.8888e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.6937e+04, 1.1980e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

# Завдання 4
Виведіть основні метрики моделі

In [268]:
preprocessor.fit(X_train)
preprocessor.transform(X_train)

array([[6.1383e+04, 9.9800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [4.5000e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.2000e+05, 1.9950e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       ...,
       [4.0909e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [6.8888e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.6937e+04, 1.1980e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [269]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
model = Pipeline(
    [("prep", preprocessor),
     ("model", DecisionTreeRegressor(max_depth = 20))
         ]
)

model


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=20))])

In [270]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=20))])

In [271]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  #df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [272]:
y_pred = model.predict(X_test)

In [273]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [274]:
all_metrics

,train,test
MAE,0.13,1.47
RMSE,0.49,3.24
R2,0.99,0.76


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

In [279]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
model = Pipeline(
    [("prep", preprocessor),
     ("model", DecisionTreeRegressor(max_depth = 9))
         ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=9))])

In [280]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=9))])

In [282]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  #df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [283]:
y_pred = model.predict(X_test)

In [284]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [285]:
all_metrics

,train,test
MAE,1.27,1.95
RMSE,2.14,3.44
R2,0.90,0.73


# Завдання 6
Збережіть модель

In [277]:
import joblib
joblib.dump(model, 'my_model.pkl')


['my_model.pkl']

In [278]:
load_model = joblib.load('my_model.pkl')
load_model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=20))])